<a href="https://colab.research.google.com/github/BrunoGomesCoelho/mosquito-networking/blob/master/notebooks/1.3-BrunoGomesCoelho_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab console code
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
"""

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}setInterval(ClickConnect,60000)\n'

## Check tensorflow version

In [157]:
import tensorflow as tf
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [ ]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [2]:
# Start logging time
import time
start_time = time.time()

In [74]:
COLAB_IDX = 1
TEMPERATURE = "t02"
FORMAT = "repeat"

In [4]:
TESTING = True
COLAB = False

if COLAB:
    BASE_DIR = "/content/drive/My Drive/IC/mosquito-networking/"
else:
    BASE_DIR = "../"

In [5]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [6]:
if COLAB:
    import sys
    sys.path.append("/content/drive/My Drive/IC/mosquito-networking/")

In [7]:
if COLAB:
    !python3 -m pip install -qr "/content/drive/My Drive/IC/mosquito-networking/drive_requirements.txt"

In [55]:
if not COLAB:
    # OPTIONAL: Load the "autoreload" extension so that code can change
    %load_ext autoreload
    # OPTIONAL: always reload modules so that as you change code in src, it gets loaded
    %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import numpy as np
import pandas as pd
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()



- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Experiment params

In [109]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'epochs': 2 if TESTING else 100}

# load data

In [128]:
# Load scaler
#scaler = joblib.load("../data/interim/scaler.pkl")
scaler = joblib.load(BASE_DIR + 
                     f"data/interim/scaler_{FORMAT}_{TEMPERATURE}.pkl")

In [76]:
data = np.load(BASE_DIR + f"data/interim/all_wavs_{FORMAT}_{TEMPERATURE}.npz",
               allow_pickle=True)

In [148]:
# Parse and divide data
train_data = data["train"]
val_data = data["val"]
test_data = data["test"]

x_train = scaler.transform(train_data[:, :-1])
y_train = train_data[:, -1]

x_val = scaler.transform(val_data[:, :-1])
y_val = val_data[:, -1]

x_test = scaler.transform(test_data[:, :-1])
y_test = test_data[:, -1]

In [149]:
# Reshape data
x_train = x_train.reshape((*x_train.shape, 1))
x_val = x_val.reshape((*x_val.shape, 1))
x_test = x_test.reshape((*x_test.shape, 1))

In [150]:
# Generators
# I am not using generators since currently the dataset fits on RAM

# Load model

In [213]:
from src.models.wingbeats_model import WingbeatsNetModel
model = WingbeatsNetModel()
# load weights?
#f"runs/colab/{COLAB_IDX-1}/model_epoch_90.pt"))
model.summary() 

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_77 (Conv1D)           (None, 11023, 32)         128       
_________________________________________________________________
global_average_pooling1d_15  (None, 32)                0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


![](wingbeats_model.png)

# Start tensorboard

In [0]:
save_path =  BASE_DIR + f"runs/TCC/{TEMPERATURE}/{COLAB_IDX}/"

# train function

In [152]:
MODEL = "testing"
model_name = f"{MODEL}_{FORMAT}"

In [161]:
from tensorflow import  keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                        ReduceLROnPlateau, CSVLogger)

In [190]:
temp = x_train.copy()

In [192]:
temp_y = y_train.copy()

In [194]:
temp.shape

(2972, 11025, 1)

In [197]:
x_train = x_train[:20]
y_train = y_train[:20]

In [209]:
params["epochs"] = 100
params["batch_size"] = 1

In [ ]:
from datetime import datetime
import keras

top_weights_path = 'model_' + model_name + '.h5'

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks_list = [
    #ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1,
    #                save_best_only=True, save_weights_only=True),
    #EarlyStopping(monitor='val_accuracy', patience=6, verbose=1),
    tensorboard_callback]#,
    #ReduceLROnPlateau(monitor = 'val_accuracy', factor=0.1, 
    #                  patience=3, verbose=1),
    #CSVLogger('model_' + model_name + '.log')]

histories = model.fit(x_train, y_train, 
          validation_data=[x_val, y_val],
          callbacks=callbacks_list,
          **params)

#model.load_weights(top_weights_path)
#loss, acc = model.evaluate(x_test, y_test)

#print('loss', loss)
#print('Test accuracy:', acc)

Train on 20 samples, validate on 743 samples
Epoch 1/100
20/20 [==============================] - 2s 83ms/step - loss: 0.7017 - accuracy: 0.4500 - val_loss: 0.6932 - val_accuracy: 0.5141
Epoch 2/100
20/20 [==============================] - 2s 78ms/step - loss: 0.6850 - accuracy: 0.6000 - val_loss: 0.6937 - val_accuracy: 0.4926
Epoch 3/100
20/20 [==============================] - 2s 75ms/step - loss: 0.6842 - accuracy: 0.6500 - val_loss: 0.6940 - val_accuracy: 0.4926
Epoch 4/100
20/20 [==============================] - 2s 75ms/step - loss: 0.6796 - accuracy: 0.5500 - val_loss: 0.6945 - val_accuracy: 0.4926
Epoch 5/100
20/20 [==============================] - 2s 76ms/step - loss: 0.6893 - accuracy: 0.5500 - val_loss: 0.6953 - val_accuracy: 0.4926
Epoch 6/100
20/20 [==============================] - 1s 75ms/step - loss: 0.6901 - accuracy: 0.6000 - val_loss: 0.6960 - val_accuracy: 0.4926
Epoch 7/100
13/20 [==================>...........] - ETA: 0s - loss: 0.6915 - accuracy: 0.5385    

In [188]:
loss, acc

(0.694406767597113, 0.49775785207748413)